In [57]:
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from shapely.geometry import shape, Point, Polygon
import re, json

In [58]:
regex = re.compile('[^a-zA-Z ]')
stops = [word.encode('ascii') for word in stopwords.words("english")]
lemmatizer = WordNetLemmatizer()
#Set resturant as food, set sport area as sport, set beach eara as beach
food = wn.synset('food.n.01')
sport = wn.synset('sport.n.01')
park = wn.synset('park.n.01')

In [59]:
#Sport area
AbPark=Polygon([(-37.842048, 144.958999),(-37.834353, 144.972602),(-37.856771, 144.982184),(-37.859224, 144.978155)
        ,(-37.850783, 144.966673)])

Vincent=Polygon([(-37.839150, 144.952463),(-37.837871, 144.957055),(-37.838812, 144.957484),(-37.840151, 144.952828)])
Skate=Point(-37.875667, 144.975475).buffer(0.00650)


stadiums = [AbPark,Vincent,Skate]

In [60]:
#Restruant
QuestRest=Polygon([(-37.836557, 144.973283),(-37.836125, 144.975482),(-37.844058, 144.978246),(-37.844503, 144.976234)])
easterncafe=Point(-37.837166, 144.967262).buffer(0.000250)
CopperSmith =Point(-37.837822, 144.963080).buffer(0.000100)
Emerald=Point(-37.837117, 144.962864).buffer(0.000100)
Lamos = Point(-37.838511, 144.960445).buffer(0.000100)
Montague = Point(-37.837377, 144.954176).buffer(0.000200)
Birdportres=Polygon([(-37.840622, 144.953489),(-37.839859, 144.956482),
                     (-37.840791, 144.956858),(-37.841460, 144.956010),
                     (-37.841725, 144.953902)])
Taco = Point(-37.850153, 144.964556).buffer(0.000500)
Pierrest = Point(-37.864840, 144.965657).buffer(0.000346)

restaurants = [QuestRest,easterncafe,CopperSmith,Emerald,Lamos,Montague,Birdportres,Taco,Pierrest]

In [61]:
#Beach
Stkildabatch = Point(-37.865126, 144.972736).buffer(0.001150)
beachst = Polygon([(-37.866960, 144.972780),(-37.866368, 144.974140),(-37.870943, 144.976280),(-37.870951, 144.975304)])
beach=[Stkildabatch,beachst]

In [62]:
data = "I like park and beach"
get_ranks(data,"park")

TypeError: string indices must be integers

In [63]:
aa = wn.synset('park' + '.n.01')
print(park.path_similarity(aa))

1.0


In [64]:
def get_ranks(data,place):
    if place == None:
        return
    text = data['doc']['text']
    text = regex.sub('', text)
    text = re.sub( '\s+', ' ', text ).strip()
    filtered_text = [word for word in text.split(' ') if word not in stops]
    lemmatized_text = [lemmatizer.lemmatize(word)  for word in filtered_text]
    if len(text) > 0:
        ranks = [0.0,0.0,0.0]
        for word in lemmatized_text:
            try:
                test = wn.synset(word.lower() + '.n.01')
                f = food.path_similarity(test)
                s = sport.path_similarity(test)
                b = park.path_similarity(test)
                if f > 0.1:
                    ranks[0] = ranks[0] + f
                if s > 0.1:
                    ranks[1] = ranks[1] + s
                if b > 0.1:
                    ranks[2] = ranks[2] + b
            except Exception as e:
                pass
        ranks = [i * 10 for i in ranks]
        ranks = [int(i) for i in ranks]
        ranks = ranks + [place]
        print(ranks)
        if any(i != 0 for i in ranks[:3]):
            return (',').join(map(str,ranks)) + '\n'

In [65]:
def check_zone(point):
    for shape in stadiums:
        if shape.contains(point):
            return "statium"
    for shape in beach:
        if shape.contains(point):
            return "park"
    for shape in restaurants:
        if shape.contains(point):
            return "restaurant"

In [66]:
import pandas as pd
geotwitter = pd.read_json('geoTwitter.json')

In [67]:
with open('train_data.txt','w') as train:
    for index, row in geotwitter.iterrows():
        try:
            data = row[2]
            cord = data['value']['geometry']['coordinates'].copy()
            cord.reverse()
            point=Point(*tuple(cord))
            train.write(get_ranks(data,check_zone(point)))
        except Exception as e:
            pass

[1, 0, 0, 'statium']
[2, 4, 0, 'statium']
[3, 1, 0, 'statium']
[0, 0, 0, 'statium']
[1, 0, 0, 'statium']
[0, 2, 0, 'statium']
[2, 1, 0, 'statium']
[0, 1, 0, 'statium']
[1, 0, 0, 'statium']
[1, 1, 2, 'statium']
[1, 1, 2, 'statium']
[5, 2, 2, 'statium']
[0, 0, 0, 'statium']
[0, 0, 0, 'statium']
[0, 0, 0, 'statium']
[1, 0, 0, 'statium']
[3, 0, 0, 'statium']
[5, 4, 2, 'statium']
[0, 0, 0, 'statium']
[3, 0, 1, 'statium']
[2, 0, 2, 'statium']
[0, 0, 0, 'statium']
[11, 0, 0, 'statium']
[2, 0, 1, 'statium']
[2, 0, 1, 'statium']
[2, 1, 0, 'statium']
[3, 1, 2, 'statium']
[2, 0, 1, 'statium']
[6, 4, 1, 'statium']
[4, 0, 0, 'statium']
[3, 6, 1, 'statium']
[2, 0, 1, 'statium']
[1, 2, 1, 'statium']
[1, 0, 0, 'statium']
[1, 3, 0, 'statium']
[3, 1, 0, 'statium']
[2, 0, 0, 'statium']
[0, 0, 0, 'statium']
[1, 2, 1, 'statium']
[3, 1, 0, 'statium']
[1, 1, 0, 'statium']
[0, 1, 1, 'statium']
[2, 1, 1, 'statium']
[3, 0, 1, 'statium']
[1, 2, 1, 'statium']
[1, 0, 0, 'statium']
[0, 1, 1, 'statium']
[1, 0, 0, 's

[0, 1, 0, 'park']
[0, 0, 0, 'park']
[3, 0, 0, 'park']
[2, 1, 0, 'park']
[0, 0, 2, 'park']
[4, 0, 2, 'park']
[2, 0, 1, 'park']
[1, 0, 0, 'park']
[1, 0, 0, 'park']
[12, 0, 0, 'park']
[1, 0, 0, 'park']
[4, 0, 0, 'park']
[8, 2, 2, 'park']
[0, 1, 2, 'park']
[2, 2, 0, 'park']
[0, 0, 1, 'park']
[7, 0, 0, 'park']
[1, 0, 1, 'park']
[5, 0, 1, 'park']
[0, 0, 1, 'park']
[3, 0, 0, 'park']
[1, 0, 1, 'park']
[1, 1, 1, 'park']
[5, 0, 2, 'park']
[1, 0, 0, 'park']
[2, 0, 1, 'park']
[1, 0, 1, 'park']
[1, 0, 1, 'park']
[1, 0, 1, 'park']
[1, 0, 1, 'park']
[0, 1, 0, 'park']
[2, 0, 0, 'park']
[1, 0, 1, 'park']
[9, 0, 1, 'park']
[1, 0, 0, 'park']
[1, 0, 0, 'park']
[0, 0, 1, 'park']
[1, 0, 0, 'park']
[0, 0, 0, 'park']
[2, 0, 0, 'park']
[0, 2, 0, 'park']
[0, 0, 0, 'park']
[3, 0, 3, 'park']
[4, 1, 0, 'park']
[4, 0, 0, 'park']
[0, 0, 0, 'park']
[6, 0, 1, 'park']
[5, 0, 0, 'park']
[2, 0, 0, 'park']
[5, 3, 2, 'park']
[1, 0, 2, 'park']
[3, 1, 1, 'park']
[1, 1, 0, 'park']
[2, 1, 1, 'park']
[3, 0, 0, 'park']
[7, 2, 0,

[1, 1, 2, 'park']
[1, 0, 1, 'park']
[1, 0, 1, 'park']
[3, 0, 2, 'park']
[1, 0, 1, 'park']
[2, 2, 0, 'park']
[2, 0, 2, 'park']
[1, 0, 1, 'park']
[0, 0, 0, 'park']
[2, 0, 2, 'park']
[1, 0, 1, 'park']
[0, 0, 2, 'park']
[1, 1, 1, 'park']
[4, 0, 1, 'park']
[1, 1, 0, 'park']
[3, 1, 3, 'park']
[1, 2, 2, 'park']
[2, 0, 2, 'park']
[2, 0, 1, 'park']
[1, 1, 1, 'park']
[14, 1, 0, 'park']
[2, 0, 2, 'park']
[2, 0, 1, 'park']
[4, 2, 1, 'park']
[4, 1, 2, 'park']
[1, 0, 1, 'park']
[1, 0, 1, 'park']
[1, 1, 2, 'park']
[1, 0, 1, 'park']
[1, 0, 2, 'park']
[0, 0, 0, 'park']
[2, 1, 0, 'park']
[1, 0, 0, 'park']
[5, 0, 1, 'park']
[4, 0, 4, 'park']
[1, 0, 2, 'park']
[4, 0, 3, 'park']
[1, 0, 2, 'park']
[2, 0, 2, 'park']
[2, 0, 1, 'park']
[1, 2, 0, 'park']
[3, 0, 1, 'park']
[0, 1, 0, 'park']
[2, 0, 2, 'park']
[0, 0, 1, 'park']
[3, 3, 1, 'park']
[0, 3, 2, 'park']
[2, 0, 2, 'park']
[0, 0, 0, 'park']
[1, 0, 1, 'park']
[5, 0, 0, 'park']
[1, 0, 1, 'park']
[1, 0, 1, 'park']
[1, 0, 1, 'park']
[2, 0, 2, 'park']
[1, 0, 1,

[6, 0, 2, 'park']
[2, 0, 1, 'park']
[0, 0, 0, 'park']
[2, 0, 1, 'park']
[2, 5, 1, 'park']
[1, 0, 2, 'park']
[5, 0, 1, 'park']
[1, 0, 1, 'park']
[1, 3, 0, 'park']
[3, 0, 6, 'park']
[2, 0, 1, 'park']
[1, 1, 1, 'park']
[2, 0, 0, 'park']
[1, 0, 1, 'park']
[3, 0, 0, 'park']
[0, 0, 0, 'park']
[7, 0, 2, 'park']
[1, 1, 1, 'park']
[2, 0, 1, 'park']
[0, 0, 0, 'park']
[1, 0, 2, 'park']
[1, 0, 1, 'park']
[1, 2, 1, 'park']
[1, 1, 0, 'park']
[1, 0, 2, 'park']
[1, 3, 0, 'park']
[1, 0, 0, 'park']
[2, 1, 1, 'park']
[0, 1, 1, 'park']
[1, 1, 1, 'park']
[2, 0, 2, 'park']
[1, 0, 1, 'park']
[1, 0, 1, 'park']
[1, 0, 1, 'park']
[1, 0, 1, 'park']
[1, 0, 1, 'park']
[1, 0, 1, 'park']
[1, 0, 1, 'park']
[1, 0, 1, 'park']
[1, 0, 1, 'park']
[1, 0, 2, 'park']
[1, 3, 0, 'park']
[2, 0, 3, 'park']
[3, 0, 3, 'park']
[1, 0, 2, 'park']
[1, 0, 4, 'park']
[2, 0, 1, 'park']
[2, 0, 1, 'park']
[1, 0, 2, 'park']
[2, 0, 1, 'park']
[4, 1, 2, 'park']
[2, 3, 2, 'park']
[3, 0, 0, 'park']
[1, 0, 3, 'park']
[1, 0, 3, 'park']
[0, 0, 0, 

[4, 2, 2, 'park']
[0, 0, 1, 'park']
[2, 0, 2, 'park']
[1, 0, 6, 'park']
[1, 0, 1, 'park']
[1, 0, 1, 'park']
[0, 0, 2, 'park']
[1, 0, 1, 'park']
[1, 0, 2, 'park']
[4, 0, 2, 'park']
[4, 0, 1, 'park']
[1, 0, 1, 'park']
[1, 2, 1, 'park']
[3, 0, 2, 'park']
[1, 1, 1, 'park']
[1, 0, 2, 'park']
[1, 0, 1, 'park']
[0, 0, 0, 'park']
[0, 0, 0, 'park']
[0, 1, 0, 'park']
[2, 0, 1, 'park']
[1, 1, 0, 'park']
[2, 0, 0, 'park']
[5, 0, 3, 'park']
[1, 0, 1, 'park']
[1, 1, 1, 'park']
[2, 2, 0, 'park']
[1, 1, 2, 'park']
[1, 0, 1, 'park']
[2, 0, 3, 'park']
[1, 0, 1, 'park']
[0, 1, 0, 'park']
[6, 1, 2, 'park']
[1, 0, 0, 'park']
[1, 0, 4, 'park']
[1, 0, 1, 'park']
[2, 0, 2, 'park']
[8, 0, 1, 'park']
[1, 0, 2, 'park']
[4, 0, 3, 'park']
[5, 1, 2, 'park']
[3, 0, 0, 'park']
[1, 0, 1, 'park']
[2, 3, 2, 'park']
[1, 0, 1, 'park']
[6, 0, 5, 'park']
[2, 0, 3, 'park']
[3, 0, 2, 'park']
[5, 1, 4, 'park']
[3, 0, 1, 'park']
[1, 0, 1, 'park']
[0, 0, 2, 'park']
[1, 0, 1, 'park']
[1, 0, 0, 'park']
[1, 0, 0, 'park']
[1, 0, 1, 

...

[2, 0, 1, 'statium']
[2, 1, 0, 'statium']
[4, 1, 1, 'statium']
[2, 0, 1, 'statium']
[1, 0, 1, 'statium']
[3, 3, 1, 'statium']
[0, 0, 1, 'statium']
[1, 1, 1, 'statium']
[1, 2, 0, 'statium']
[2, 0, 0, 'statium']
[1, 0, 1, 'statium']
[1, 0, 1, 'statium']
[2, 0, 0, 'statium']
[4, 10, 2, 'statium']
[1, 0, 0, 'statium']
[2, 0, 0, 'statium']
[2, 0, 0, 'statium']
[1, 0, 0, 'statium']
[1, 0, 0, 'statium']
[1, 0, 0, 'statium']
[1, 1, 0, 'statium']
[2, 2, 0, 'statium']
[0, 1, 0, 'statium']
[1, 0, 0, 'statium']
[2, 1, 0, 'statium']
[1, 1, 0, 'statium']
[2, 2, 0, 'statium']
[1, 2, 0, 'statium']
[1, 0, 0, 'statium']
[1, 1, 0, 'statium']
[1, 2, 0, 'statium']
[2, 0, 0, 'statium']
[1, 0, 0, 'statium']
[1, 2, 1, 'statium']
[0, 2, 0, 'statium']
[2, 3, 0, 'statium']
[0, 1, 0, 'statium']
[0, 4, 0, 'statium']
[1, 4, 0, 'statium']
[2, 1, 0, 'statium']
[1, 2, 1, 'statium']
[6, 7, 1, 'statium']
[1, 2, 1, 'statium']
[0, 0, 0, 'statium']
[0, 1, 1, 'statium']
[1, 0, 0, 'statium']
[1, 0, 0, 'statium']
[1, 0, 0, 's

[2, 0, 11, 'restaurant']
[2, 0, 10, 'restaurant']
[2, 0, 0, 'restaurant']
[1, 0, 0, 'restaurant']
[4, 8, 1, 'restaurant']
[4, 2, 0, 'restaurant']
[5, 0, 0, 'restaurant']
[2, 0, 0, 'restaurant']
[2, 0, 11, 'restaurant']
[0, 2, 0, 'restaurant']
[0, 0, 11, 'restaurant']
[6, 3, 1, 'restaurant']
[3, 0, 1, 'restaurant']
[4, 0, 1, 'restaurant']
[4, 1, 1, 'restaurant']
[1, 0, 11, 'restaurant']
[0, 0, 1, 'restaurant']
[1, 0, 0, 'restaurant']
[0, 2, 0, 'restaurant']
[4, 1, 0, 'restaurant']
[0, 0, 10, 'restaurant']
[4, 1, 2, 'restaurant']
[2, 0, 1, 'restaurant']
[4, 0, 11, 'restaurant']
[1, 0, 1, 'restaurant']
[0, 0, 0, 'restaurant']
[5, 0, 10, 'restaurant']
[13, 0, 0, 'restaurant']
[4, 0, 12, 'restaurant']
[0, 0, 0, 'restaurant']
[0, 0, 10, 'restaurant']
[0, 0, 10, 'restaurant']
[10, 1, 0, 'restaurant']
[2, 1, 0, 'restaurant']
[4, 0, 10, 'restaurant']
[5, 0, 2, 'restaurant']
[1, 2, 1, 'restaurant']
[1, 2, 1, 'restaurant']
[0, 0, 10, 'restaurant']
[0, 0, 11, 'restaurant']
[3, 0, 11, 'restaurant']

[3, 0, 0, 'statium']
[4, 1, 10, 'statium']
[5, 0, 0, 'statium']
[0, 0, 0, 'statium']
[3, 0, 1, 'statium']
[1, 0, 1, 'statium']
[2, 1, 1, 'statium']
[11, 0, 0, 'statium']
[1, 0, 0, 'statium']
[1, 0, 12, 'statium']
[3, 0, 3, 'statium']
[1, 1, 0, 'statium']
[4, 1, 11, 'statium']
[4, 0, 1, 'statium']
[1, 0, 10, 'statium']
[1, 0, 10, 'statium']
[1, 0, 2, 'statium']
[0, 0, 11, 'statium']
[0, 0, 10, 'statium']
[1, 0, 1, 'statium']
[3, 0, 2, 'statium']
[1, 0, 10, 'statium']
[0, 0, 10, 'statium']
[0, 0, 0, 'statium']
[0, 0, 0, 'statium']
[0, 1, 2, 'statium']
[2, 1, 3, 'statium']
[0, 0, 2, 'statium']
[0, 0, 11, 'statium']
[8, 0, 0, 'statium']
[0, 0, 10, 'statium']
[3, 0, 1, 'statium']
[0, 2, 10, 'statium']
[0, 3, 10, 'statium']
[1, 0, 21, 'statium']
[0, 0, 12, 'statium']
[0, 3, 1, 'statium']
[1, 1, 0, 'statium']
[4, 1, 10, 'statium']
[0, 0, 10, 'statium']
[2, 0, 0, 'statium']
[2, 2, 0, 'statium']
[1, 0, 0, 'statium']
[3, 1, 4, 'statium']
[0, 0, 10, 'statium']
[1, 2, 0, 'statium']
[5, 1, 3, 'stat

[1, 0, 10, 'statium']
[2, 0, 1, 'statium']
[1, 0, 11, 'statium']
[2, 1, 1, 'statium']
[1, 0, 10, 'statium']
[1, 0, 0, 'statium']
[6, 10, 1, 'statium']
[3, 0, 11, 'statium']
[1, 2, 11, 'statium']
[0, 1, 2, 'statium']
[2, 0, 0, 'statium']
[1, 2, 11, 'statium']
[1, 0, 1, 'statium']
[3, 0, 11, 'statium']
[1, 0, 10, 'statium']
[1, 0, 10, 'statium']
[0, 0, 0, 'statium']
[1, 0, 10, 'statium']
[4, 1, 1, 'statium']
[6, 0, 0, 'statium']
[4, 0, 10, 'statium']
[2, 5, 0, 'statium']
[0, 1, 0, 'statium']
[6, 0, 0, 'statium']
[2, 2, 0, 'statium']
[1, 0, 11, 'statium']
[2, 1, 0, 'statium']
[1, 1, 10, 'statium']
[1, 2, 0, 'statium']
[2, 0, 2, 'statium']
[0, 2, 0, 'statium']
[3, 2, 0, 'statium']
[5, 0, 2, 'statium']
[1, 0, 10, 'statium']
[4, 3, 0, 'statium']
[3, 4, 1, 'statium']
[1, 0, 11, 'statium']
[4, 0, 21, 'statium']
[2, 1, 20, 'statium']
[3, 0, 20, 'statium']
[3, 0, 20, 'statium']
[3, 0, 20, 'statium']
[8, 0, 1, 'statium']
[4, 0, 1, 'statium']
[6, 0, 0, 'statium']
[9, 1, 0, 'statium']
[6, 2, 0, 'st

[1, 1, 1, 'restaurant']
[5, 0, 0, 'statium']
[3, 0, 1, 'statium']
[0, 0, 1, 'statium']
[1, 4, 0, 'statium']
[1, 5, 0, 'restaurant']
[0, 0, 0, 'restaurant']
[0, 0, 0, 'restaurant']
[3, 2, 0, 'restaurant']
[0, 0, 1, 'statium']
[1, 1, 1, 'statium']
[1, 2, 2, 'statium']
[0, 0, 0, 'statium']
[2, 0, 12, 'statium']
[4, 2, 0, 'statium']
[0, 2, 1, 'statium']
[0, 0, 0, 'statium']
[2, 1, 4, 'statium']
[2, 0, 2, 'statium']
[2, 0, 2, 'statium']
[2, 0, 2, 'statium']
[3, 2, 0, 'statium']
[2, 1, 0, 'statium']
[2, 0, 1, 'statium']
[3, 2, 0, 'statium']
[3, 2, 1, 'statium']
[3, 1, 0, 'statium']
[2, 0, 2, 'statium']
[4, 0, 2, 'statium']
[2, 0, 2, 'statium']
[5, 4, 0, 'restaurant']
[9, 5, 2, 'restaurant']
[5, 0, 0, 'restaurant']
[0, 2, 0, 'restaurant']
[4, 0, 1, 'restaurant']
[4, 0, 3, 'restaurant']
[3, 4, 1, 'restaurant']
[2, 0, 1, 'restaurant']
[3, 3, 1, 'restaurant']
[3, 0, 0, 'restaurant']
[0, 1, 1, 'restaurant']
[1, 1, 1, 'restaurant']
[3, 2, 2, 'restaurant']
[2, 1, 1, 'restaurant']
[11, 1, 2, 'restau